### Step1 : Reading Data    

In [1]:
import pandas as pd
pdf = pd.read_csv("/workspaces/codespaces-jupyter/data/labelled_newscatcher_dataset.csv",sep=";")
pdf["id"] = pdf.index
display(pdf)

,topic,link,domain,published_date,title,lang,id
0,SCIENCE,https://www.eurekalert.org/pub_releases/2020-0...,eurekalert.org,2020-08-06 13:59:45,A closer look at water-splitting's solar fuel ...,en,0
1,SCIENCE,https://www.pulse.ng/news/world/an-irresistibl...,pulse.ng,2020-08-12 15:14:19,"An irresistible scent makes locusts swarm, stu...",en,1
2,SCIENCE,https://www.express.co.uk/news/science/1322607...,express.co.uk,2020-08-13 21:01:00,Artificial intelligence warning: AI will know ...,en,2
3,SCIENCE,https://www.ndtv.com/world-news/glaciers-could...,ndtv.com,2020-08-03 22:18:26,Glaciers Could Have Sculpted Mars Valleys: Study,en,3
4,SCIENCE,https://www.thesun.ie/tech/5742187/perseid-met...,thesun.ie,2020-08-12 19:54:36,Perseid meteor shower 2020: What time and how ...,en,4
...,...,...,...,...,...,...,...
108769,NATION,https://www.vanguardngr.com/2020/08/pdp-govern...,vanguardngr.com,2020-08-08 02:40:00,PDP governors’ forum urges security agencies t...,en,108769
108770,BUSINESS,https://www.patentlyapple.com/patently-apple/2...,patentlyapple.com,2020-08-08 01:27:12,"In Q2-20, Apple Dominated the Premium Smartpho...",en,108770
108771,HEALTH,https://www.belfastlive.co.uk/news/health/coro...,belfastlive.co.uk,2020-08-12 17:01:00,Coronavirus Northern Ireland: Full breakdown s...,en,108771
108772,ENTERTAINMENT,https://www.thenews.com.pk/latest/696364-paul-...,thenews.com.pk,2020-08-05 04:59:00,Paul McCartney details post-Beatles distress a...,en,108772


In [2]:
from sentence_transformers import InputExample

pdf_subset = pdf.head(1000)

def example_create_fn(doc1:pd.Series) -> InputExample:
    """
        Helper Function That Outputs a Sentence_transformer guid , label , and text
    """
    return InputExample(texts=[doc1])

faiss_train_examples = pdf_subset.apply(lambda x:example_create_fn(x["title"]),axis=1).tolist()

## Step 2 : Vectorize Text Into Embedding Vectors
> SentenceTransformer(all-MiniLM-L6-v2) : This is a sentence-transformers model: It maps sentences & paragraphs to a 384 dimensional dense vector space 
>                      and can be used for tasks like clustering or semantic search.

In [4]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(
    "all-MiniLM-L6-v2"
)
faiss_title_embedding = model.encode(pdf_subset.title.values.tolist())
len(faiss_title_embedding),faiss_title_embedding.shape

(1000, (1000, 384))

## **Step 3 : Saving Embedding Vectors to FAISS Index**
> \# *Create an FAISS Index Object , based on Embedding vectors ==> Normalie Vectors ==> Add These Vectors to FAISS Index.*

In [5]:
import numpy as np 
import faiss

pdf_to_index = pdf_subset.set_index(["id"],drop=False)
id_index = np.array(pdf_to_index.id.values).flatten().astype("int")

content_encoded_normalzed = faiss_title_embedding.copy()
faiss.normalize_L2(content_encoded_normalzed)

index_content = faiss.IndexIDMap(faiss.IndexFlatIP(len(faiss_title_embedding[0])))
index_content.add_with_ids(content_encoded_normalzed,id_index)

## **Step 4: Search For relevant documents**
> Vectorize our Query Text , then Search for the vectors with closest distance

In [6]:
def search_content(query,pdf_to_index,k=3):
    query_vector = model.encode([query])
    faiss.normalize_L2(query_vector)

    top_k = index_content.search(query_vector,k)
    ids = top_k[1][0].tolist()
    similarities = top_k[0][0].tolist()
    results = pdf_to_index.loc[ids]
    results["similarities"] = similarities
    return results

In [7]:
display(search_content("animal",pdf_to_index))

,topic,link,domain,published_date,title,lang,id,similarities
id,,,,,,,,
176,TECHNOLOGY,https://www.pushsquare.com/news/2020/08/random...,pushsquare.com,2020-08-03 16:30:00,Random: You Can Pick Up and Pet Cats in Assass...,en,176,0.391902
975,HEALTH,https://www.news-medical.net/news/20200813/Res...,news-medical.net,2020-08-13 05:18:00,Researchers explore social behavior of animals...,en,975,0.376784
99,TECHNOLOGY,https://www.gematsu.com/2020/08/ghostwire-toky...,gematsu.com,2020-08-07 16:43:13,Ghostwire: Tokyo confirms dog petting,en,99,0.344059
